In [14]:
import numpy as np
from scipy.ndimage import gaussian_filter
import tqdm
from tqdm import trange, tqdm

n=np.arange(7)
Rsall=1/2*np.sqrt(2)**n

In [36]:
nd=np.load("/data/dell5/userdir/heyi/EAGLE/5.21_gu_256.npy")
for i in trange(7):
    Rs=1/2*np.sqrt(2)**i
    N=256#网格数目
    #空网格加上平均数密度
    ave=np.sum(nd)/N/N/N
    nd=(nd-ave)/ave

    deltk=np.fft.fftn(nd) #傅里叶转换

    s=1.5*(0.307-0.0455)*0.6777**2*10**4
    v=2*np.pi/67.77            #2*pi/L L为box长度
    a=np.fft.fftfreq(N,1./N)

    nx=a[:,None,None]+np.zeros_like(a[None,:,None])+np.zeros_like(a[None,None,:])
    ny=a[None,:,None]+np.zeros_like(a[:,None,None])+np.zeros_like(a[None,None,:])
    nz=a[None,None,:]+np.zeros_like(a[:,None,None])+np.zeros_like(a[None,:,None])

    deltk=deltk/(np.sinc(nx/N)*np.sinc(ny/N)*np.sinc(nz/N))**2

    k=np.sqrt((v*nx)**2+(v*ny)**2+(v*nz)**2)
    deltk=deltk*np.exp(-0.5*k*k*Rs**2)

    fik=s*deltk/(-k**2)

    #fik=fik*np.exp(-0.5*k*k*100)#直接求T 二阶偏导
    fik[0,0,0]=0.0

    h11=-fik*v*nx*v*nx
    h12=h21=-fik*v*nx*v*ny
    h13=h31=-fik*v*nx*v*nz
    h22=-fik*v*ny*v*ny
    h23=h32=-fik*v*ny*v*nz
    h33=-fik*v*nz*v*nz

    h11=np.fft.ifftn(h11)
    h12=h21=np.fft.ifftn(h12)
    h13=h31=np.fft.ifftn(h13)
    h22=np.fft.ifftn(h22)
    h23=h32=np.fft.ifftn(h23)
    h33=np.fft.ifftn(h33)

    h11=h11.reshape(N**3,1).real
    h12=h21=h12.reshape(N**3,1).real
    h13=h31=h13.reshape(N**3,1).real
    h22=h22.reshape(N**3,1).real
    h23=h32=h23.reshape(N**3,1).real
    h33=h33.reshape(N**3,1).real


    ten=np.hstack((h11,h12,h13,h21,h22,h23,h31,h32,h33)).reshape(-1,3,3)
    ten=np.array(ten,dtype=np.float32)
    lm,l=np.linalg.eig(ten)
    lmdn=np.ones((N**3,12),dtype=np.float32)
    lmdn[:,0]=lm[:,0]
    lmdn[:,1:4]=l[:,:,0]
    lmdn[:,4]=lm[:,1]
    lmdn[:,5:8]=l[:,:,1]
    lmdn[:,8]=lm[:,2]
    lmdn[:,9:12]=l[:,:,2]


    s=1.5*(0.307-0.0455)*0.6777**2*10**4
  
    eig=lmdn
    eig[:,0]=eig[:,0]/s
    eig[:,4]=eig[:,4]/s
    eig[:,8]=eig[:,8]/s

    print (1)
    lmd=np.zeros((N**3,3),dtype=np.float)
    n1=np.zeros((N**3,3),dtype=np.float)
    n2=np.zeros((N**3,3),dtype=np.float)
    n3=np.zeros((N**3,3),dtype=np.float)

    lmd[:,0]=eig[:,0]
    lmd[:,1]=eig[:,4]
    lmd[:,2]=eig[:,8]

    print(2) 
    n1[:,0:3]=eig[:,1:4]
    n2[:,0:3]=eig[:,5:8]
    n3[:,0:3]=eig[:,9:12]
    print(3)
    Rsstr=np.str(Rs)
    np.save('/data/dell5/userdir/hy/EAGLE_NEXUSw_256_'+Rsstr+'smooth_200911.npy',lmd)
    np.save('/data/dell5/userdir/hy/EAGLE_NEXUSv_256_'+Rsstr+'smooth_200911.npy',v)    

/home/heyi/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:27: RuntimeWarning: divide by zero encountered in true_divide
/home/heyi/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:27: RuntimeWarning: invalid value encountered in true_divide


In [37]:
a=lmd>Rth

print(4)
b=a[:,0]*2**2+a[:,1]*2+a[:,2]*1
c=(b==7)*3
f=((b==6)|(b==5)|(b==3))*2
s=((b==4)|(b==2)|(b==1))*1
v=(b==0)*0

fil=c+f+s+v
print(5)
filn1=(b==6).reshape(-1,1)*n3
filn2=(b==5).reshape(-1,1)*n2
filn3=(b==3).reshape(-1,1)*n1
filn=filn1[:,0:3]+filn2[:,0:3]+filn3[:,0:3]
print(6)


1
2
3
4
5
6


In [38]:
lmd=lmd.reshape(N,N,N,3)
v=np.zeros([N,N,N,3,3])
v[0:N,0:N,0:N,0,0:3]=n1.reshape(N,N,N,3)
v[0:N,0:N,0:N,1,0:3]=n2.reshape(N,N,N,3)
v[0:N,0:N,0:N,2,0:3]=n3.reshape(N,N,N,3)

In [39]:
fil=fil.reshape(N,N,N)
c=fil
voidpos=np.where(c==0)
sheetpos=np.where(c==1)
filpos=np.where(c==2)
clusterpos=np.where(c==3)

-996461445766118.2 122547091643268.2 449998866902235.3 423915487220614.9


In [53]:
np.save('/data/dell5/userdir/hy/EAGLEfil_256_1.6smooth_0.8Rs_heyi200910.npy',fil)

In [54]:
np.save('/data/dell5/userdir/hy/EAGLEw_256_1.6smooth_0.8Rs_heyi200910.npy',lmd)
np.save('/data/dell5/userdir/hy/EAGLEv_256_1.6smooth_0.8Rs_heyi200910.npy',v)